In [ ]:
!pip install opencv-python

In [1]:
from IPython import display
import ultralytics
from ultralytics import YOLO
import cv2

In [2]:
model = YOLO('./runs/segment/train8/weights/best.pt')

In [ ]:
# test video 
import cv2
from ultralytics import YOLO

# model = YOLO("best.pt")
cap = cv2.VideoCapture('./video/general-item-2.mp4')

desired_fps = 25
frame_delay = int(1000/desired_fps)# ใช้ในการควบคุมความเร็วในการเล่นวิดีโอ

# เมื่อเปิดไฟล์วิดีโอไม่ได้
if not cap.isOpened():
    print("Error: file not open")
    exit()

img_counter = 0 # ไว้บันทึกรูปภาพตามครั้งที่กด spacebar

# ลูปรูปวิดีโอในแต่ละเฟรม
while True:
    success, frame = cap.read()

    if success:
        results = model(frame)

        # สร้างเฟรมที่มีขอบเขตและป้ายกำกับ ลงไปบนภาพ
        annotated_frame = results[0].plot()

        #  แสดงผล
        cv2.imshow("General model", annotated_frame)

        k = cv2.waitKey(frame_delay)# log speed
        if k & 0xFF == ord("q"):# เมื่อกด q จะทำการปิดวิดีโอ
            break
        elif k%256 == 32: # spacebar trigger
            img_name = "frame_save{}.png".format(img_counter)
            cv2.imwrite(img_name, annotated_frame) 
            img_counter+=1
            print("{} written!".format(img_name))
            frame = cv2.putText(frame,
                                "img :{} save!".format(img_name),
                                org = (50, 50),
                                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                fontScale=1,
                                color = (255,0,0),
                                thickness=2,
                                lineType=cv2.LINE_AA
            )
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# test video and save
import cv2
from ultralytics import YOLO

# โหลดโมเดล
# model = YOLO("best.pt")
cap = cv2.VideoCapture('./video/general-item-1.mp4')

desired_fps = 25
frame_delay = int(1000 / desired_fps)  # ควบคุมความเร็วในการเล่นวิดีโอ

# ตรวจสอบว่าเปิดไฟล์วิดีโอได้หรือไม่
if not cap.isOpened():
    print("Error: file not open")
    exit()

# กำหนดค่าการบันทึกวิดีโอ
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # ใช้โค้ด 'mp4v' สำหรับไฟล์ .mp4
out = cv2.VideoWriter('output_video.mp4', fourcc, desired_fps, (width, height))

img_counter = 0  # ใช้สำหรับนับจำนวนการบันทึกภาพ

# วนลูปเพื่ออ่านแต่ละเฟรม
while True:
    success, frame = cap.read()

    if success:
        # ทำการทำนายผล
        results = model(frame)
        
        # สร้างเฟรมที่มีขอบเขตและป้ายกำกับบนภาพ
        annotated_frame = results[0].plot()
        
        # บันทึกเฟรมที่ผ่านการ predict ลงในวิดีโอ
        out.write(annotated_frame)

        # แสดงผล
        cv2.imshow("General model", annotated_frame)

        k = cv2.waitKey(frame_delay)  # ตั้งความเร็วในการเล่นวิดีโอ
        if k & 0xFF == ord("q"):  # เมื่อกด q จะปิดวิดีโอ
            break
        elif k % 256 == 32:  # spacebar trigger
            img_name = "frame_save{}.png".format(img_counter)
            cv2.imwrite(img_name, annotated_frame)
            img_counter += 1
            print("{} written!".format(img_name))
            frame = cv2.putText(
                frame,
                "img :{} save!".format(img_name),
                org=(50, 50),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1,
                color=(255, 0, 0),
                thickness=2,
                lineType=cv2.LINE_AA
            )
    else:
        break

# ปิดการใช้งาน VideoWriter และ VideoCapture
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
# test webcam camera
from ultralytics import YOLO
import cv2

# โหลดโมเดล YOLO
# model = YOLO('best.pt')
cap = cv2.VideoCapture(0)
# กำหนดค่าการบันทึกวิดีโอ
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # ใช้โค้ด 'mp4v' สำหรับไฟล์ .mp4
out = cv2.VideoWriter('realtime_video.mp4', fourcc, desired_fps, (width, height))
img_counter = 0

# วนลูปเพื่อดูวิดีโอแต่ละเฟรม
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    frame = cv2.resize(frame, (340, 240))
    frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = model(frame)  # ทำการทำนาย (predict)
    
    if results:
        # ตรวจสอบ bounding boxes และวาดกรอบสี่เหลี่ยม พร้อมแสดงชื่อคลาส
        for box in results[0].boxes:
            x, y, w, h = box.xywh[0]
            x, y, w, h = int(x), int(y), int(w), int(h)
            class_id = int(box.cls[0])  # ดึงค่า class ของวัตถุ
            class_name = model.names[class_id]  # แปลง class id เป็นชื่อคลาส
            
            # วาดกรอบสี่เหลี่ยม
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            # ใส่ชื่อคลาสข้างกรอบ
            cv2.putText(frame, class_name, (x, y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        print("ไม่พบ bounding boxes ใน results")
    
    out.write(frame)    
    cv2.imshow('Webcam Player', frame)
    
    
    k = cv2.waitKey(frame_delay)
    if k & 0xff == ord('q'):
        break
    elif k == 32:  # spacebar trigger
        img_name = "frame_save{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        img_counter += 1
        print("{} written!".format(img_name))
        frame = cv2.putText(frame,
                            "img :{} save!".format(img_name),
                            org=(50, 50),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale=1,
                            color=(255, 0, 0),
                            thickness=2,
                            lineType=cv2.LINE_AA
        )
    
cap.release()
cv2.destroyAllWindows()


In [21]:
# test webcam camera
from ultralytics import YOLO
import cv2

cap = cv2.VideoCapture(0)

# กำหนดค่าการบันทึกวิดีโอ
desired_fps = 25
frame_delay = int(1000 / desired_fps)  # ใช้ควบคุมความเร็วการแสดงผล
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # ใช้โค้ด 'mp4v' สำหรับไฟล์ .mp4
out = cv2.VideoWriter('realtime_video.mp4', fourcc, desired_fps, (width, height))

img_counter = 0

# วนลูปเพื่อดูวิดีโอแต่ละเฟรม
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # ทำการทำนาย (predict)
    results = model(frame)
    
    if results:
        # ตรวจสอบ bounding boxes และวาดกรอบสี่เหลี่ยม พร้อมแสดงชื่อคลาส
        for box in results[0].boxes:
            x, y, w, h = box.xywh[0]
            x, y, w, h = int(x), int(y), int(w), int(h)
            class_id = int(box.cls[0])  # ดึงค่า class ของวัตถุ
            class_name = model.names[class_id]  # แปลง class id เป็นชื่อคลาส
            
            # วาดกรอบสี่เหลี่ยมและใส่ชื่อคลาส
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, class_name, (x, y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        print("ไม่พบ bounding boxes ใน results")
    
    # บันทึกเฟรมลงในวิดีโอ
    out.write(frame)
    
    # แสดงผล
    cv2.imshow('Webcam Player', frame)
    
    k = cv2.waitKey(frame_delay)
    if k & 0xff == ord('q'):
        break
    elif k == 32:  # spacebar trigger
        img_name = "frame_save{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        img_counter += 1
        print("{} written!".format(img_name))
        frame = cv2.putText(frame,
                            "img :{} save!".format(img_name),
                            org=(50, 50),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale=1,
                            color=(255, 0, 0),
                            thickness=2,
                            lineType=cv2.LINE_AA
        )
    
cap.release()
out.release()
cv2.destroyAllWindows()


0: 480x640 1 mask, 1 spoon, 137.5ms
Speed: 5.0ms preprocess, 137.5ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 mask, 99.0ms
Speed: 1.0ms preprocess, 99.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 glass, 1 mask, 131.9ms
Speed: 2.0ms preprocess, 131.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 mask, 94.3ms
Speed: 1.0ms preprocess, 94.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 mask, 93.6ms
Speed: 1.0ms preprocess, 93.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 mask, 90.7ms
Speed: 1.0ms preprocess, 90.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 mask, 88.9ms
Speed: 1.0ms preprocess, 88.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 glass, 1 mask, 99.3ms
Speed: 1.0ms preprocess, 99.3ms inference, 4.0ms postprocess per image at 